# Module 3: Inverse Planning

In [ ]:
using Gen
using Luxor

In [ ]:
@gen function generative_model(n_observations)
    
    observations = Array{Bool}(undef, n_observations, 4)
    
    square = @trace(uniform_discrete(0, 2), :square)
    circle = @trace(uniform_discrete(0, 2), :circle)
#     adversity = @trace(uniform_discrete(-1, 2), :adversity)
    adversity = -1

    for i=1:n_observations
        observations[i,1] = @trace(bernoulli(0.5), (:is_adversity, i))
        observations[i,2] = @trace(bernoulli(0.5), (:is_obj_present, i))
        observations[i,3] = @trace(bernoulli(0.5), (:is_square, i))
        
        reward = 0
        reward += observations[i,1]*adversity
        if observations[i,2]
            if observations[i,3]
                reward += square
            else
                reward += circle
            end
        end
        
        if reward > 0
            observations[i,4] = @trace(bernoulli(1.0), (:moved, i))
        else
            observations[i,4] = @trace(bernoulli(0.0), (:moved, i))
        end
    end
    
    return square, circle, adversity, observations
end;

In [ ]:
println(generative_model(5))

In [ ]:
function draw_scene(id::Int, is_adversity::Bool, is_obj_present::Bool, is_square::Bool, moved::Bool)
    Drawing(700, 350, "scene_"*string(id)*".png")
    origin()
    background("antiquewhite2")
    

    sethue("dodgerblue4")
    setline(8)
    fontsize(50)
    box_dim = 200
    box(0,0,box_dim,box_dim, :stroke)
    box(-box_dim,0,box_dim,box_dim, :stroke)
    box(box_dim,0,box_dim,box_dim, :stroke)
    
    robot = readpng("images/joyous_robot.png")
    placeimage(robot, -box_dim, 0; centered=true)

    if is_adversity
        sethue("palevioletred")
        p1 = Point(-50,-50)
        p2 = Point(50,50)
        line(p1, p2, :stroke)
        p1 = Point(50,-50)
        p2 = Point(-50,50)
        line(p1, p2, :stroke)
    end
    
    if is_obj_present
        if is_square
            sethue("firebrick")
            box(box_dim,0,100,100, :fill)
        else
            sethue("darkolivegreen3")
            p = Point(box_dim,0)
            circle(p,50, :fill)
        end
    end
    
    if moved
        setdash("longdashed")
        sethue("grey28")
        startpoint = Point(-box_dim, -120)
        endpoint = Point(box_dim, -120)
        c1 = Point(-100,-150)
        c2 = Point(100,-150)
        arrow(startpoint,  c1, c2, endpoint;
                linewidth=8,
                arrowheadlength = 30,
                arrowheadangle = pi/8)
    end
    
    finish()
    preview()
end;

In [ ]:
function draw_blank()
    Drawing(700, 350, "scene.png")
    background("antiquewhite2")
    finish()
    preview()
end;

In [ ]:
draw_scene(0, true, true, true, true)

In [ ]:
function do_inference(model, observations, amount_of_compute)
    n_observations = size(observations, 1)
    obs = Gen.choicemap()
    for i=1:n_observations
        obs[(:is_adversity, i)] = observations[i,1]
        obs[(:is_obj_present, i)] = observations[i,2]
        obs[(:is_square, i)] = observations[i,3]
        obs[(:moved, i)] = observations[i,4]
    end
    
    (trace, _) = Gen.importance_resampling(model, (n_observations,), obs, amount_of_compute);
    return trace
end;

In [ ]:
observations = [true  true  true  true
                true  false true  false
                false true  false true
                false true  true  true
                true  true  false false]

In [ ]:
for i=1:size(observations, 1)
    draw_blank()
    sleep(0.5)
    draw_scene(i, observations[i,1], observations[i,2],
               observations[i,3], observations[i,4])
    sleep(3)
end

In [ ]:
traces = [do_inference(generative_model, observations, 1000) for _=1:100];

square = 0.0
circ = 0.0

for trace in traces
    square += trace[:square]
    circ += trace[:circle]
end

square /= length(traces)
circ /= length(traces)

println("inferred values:")
println("square: ", square)
println("circle: ", circ)

In [ ]:
#########################

### Exercise

Constrain the start and destination points to two particular locations in the scene, and visualize the distribution on paths. Find a start point and destination point such that the distributions on paths is multimodal (i.e. there are two spatially separated paths that the agent may take). Describe the variability in the planned paths.

### Solution



We now write a simple algorithm for inferring the destination of an agent given (i) the scene, (ii) the start location of the agent, and (iii) a sequence of measured locations of the agent for each tick.

We will assume the agent starts in the lower left-hand corner.

In [ ]:
start = Point(0.1, 0.1);

We will infer the destination of the agent for the given sequence of observed locations:

In [ ]:
measurements = [
    Point(0.0980245, 0.104775),
    Point(0.113734, 0.150773),
    Point(0.100412, 0.195499),
    Point(0.114794, 0.237386),
    Point(0.0957668, 0.277711),
    Point(0.140181, 0.31304),
    Point(0.124384, 0.356242),
    Point(0.122272, 0.414463),
    Point(0.124597, 0.462056),
    Point(0.126227, 0.498338)];

We visualize this data set on top of the scene:

In [ ]:
info = Dict("start" => start, "scene" => scene, "measurements" => measurements)
viz = Viz(viz_server, joinpath(@__DIR__, "../inverse-planning/overlay-viz/dist"), info)
displayInNotebook(viz)

Next, we write a simple inference program for this task:

In [ ]:
function do_inference_agent_model(scene::Scene, dt::Float64, num_ticks::Int, planner_params::PlannerParams, start::Point,
                                  measurements::Vector{Point}, amount_of_computation::Int)
    
    # Create an "Assignment" that maps model addresses (:y, i)
    # to observed values ys[i]. We leave :slope and :intercept
    # unconstrained, because we want them to be inferred.
    observations = Gen.choicemap()
    observations[:start_x] = start.x
    observations[:start_y] = start.y
    for (i, m) in enumerate(measurements)
        observations[:meas => (i, :x)] = m.x
        observations[:meas => (i, :y)] = m.y
    end
    
    # Call importance_resampling to obtain a likely trace consistent
    # with our observations.
    (trace, _) = Gen.importance_resampling(agent_model, (scene, dt, num_ticks, planner_params), observations, amount_of_computation)
    
    return trace
end;

Below, we run this algorithm 1000 times, to generate 1000 approximate samples from the posterior distribution on the destination. The inferred destinations should appear as red dots on the map.

In [ ]:
info = Dict("measurements" => measurements, "scene" => scene, "start" => start)
viz = Viz(viz_server, joinpath(@__DIR__, "../inverse-planning/overlay-viz/dist"), info)
openInNotebook(viz)
sleep(5)
for i=1:1000
    trace = do_inference_agent_model(scene, dt, num_ticks, planner_params, start, measurements, 50)
    putTrace!(viz, i, trace_to_dict(trace))
end
displayInNotebook(viz)

### Exercise
The first argument to `PlannerParams` is the number of iterations of the RRT algorithm to use. The third argument to `PlannerParams` is the number of iterations of path refinement. These parameters affect the distribution on paths of the agent. Visualize traces of the `agent_model` for with a couple different settings of these two parameters to the path planning algorithm for fixed starting point and destination point. Try setting them to smaller values. Discuss.

### Solution

We have provided starter code.

In [ ]:
constraints = Gen.choicemap()
constraints[:start_x] = 0.1
constraints[:start_y] = 0.1;

Modify the `PlannerParams` in the two cells below.

In [ ]:
planner_params = PlannerParams(300, 3.0, 2000, 1.) # < change this line>

viz = Viz(viz_server, joinpath(@__DIR__, "../inverse-planning/grid-viz/dist"), [])
for i=1:12
    (trace, _) = Gen.generate(agent_model, (scene, dt, num_ticks, planner_params), constraints)
    putTrace!(viz, i, trace_to_dict(trace))
end
displayInNotebook(viz)

In [ ]:
planner_params = PlannerParams(300, 3.0, 2000, 1.) # < change this line>

viz = Viz(viz_server, joinpath(@__DIR__, "../inverse-planning/grid-viz/dist"), [])
for i=1:12
    (trace, _) = Gen.generate(agent_model, (scene, dt, num_ticks, planner_params), constraints)
    putTrace!(viz, i, trace_to_dict(trace))
end
displayInNotebook(viz)